In [1]:
import numpy as np
from skimage import measure
from time import time
from joblib import Parallel, delayed 
from tqdm import tqdm

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


In [2]:

IMAGES = 100
IMG_PER_P = 1

def run_sim(p, i, N):
    t0 = time()
    input_img = np.where(np.random.random( size=(N, N)) > p, 1, 0)
    t1 = time()
    all_labels = measure.label(input_img)
    t2 = time()
    unique, counts = np.unique(all_labels, return_counts=True)
    largest_ag = -1
    if len(unique) == 1 and unique[0] != 0:
        largest_ag = counts[0]
    elif len(unique) > 1:
        if unique[-1] == 0:
            largest_ag = counts[-2]
        else:
            largest_ag = counts[-1]

    t3 = time()
    return (i, p, largest_ag,  t1-t0, t2-t1, t3-t2, t3-t0)


In [ ]:
sims = {}
for L in [20, 50, 100, 200, 500, 1000, 2000]:
    t00 = time()
    IMG_PER_P = int(5e8)//int(np.power(L, 3) * IMAGES) 
    if IMG_PER_P < 1:
        IMG_PER_P = 1
    IMG_PER_P = 200
    results = Parallel(n_jobs=-1)(delayed(run_sim)(i%IMAGES/(IMAGES-1), i, L) for i in tqdm(range(IMAGES*IMG_PER_P), desc="L: %5d"%L)) # n_jobs = -1 means use all cores.

    sims[L] = results

L:  2000:   0%|          | 16/20000 [00:00<09:31, 34.99it/s]/Users/David/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
L:  2000:   7%|▋         | 1376/20000 [01:18<18:11, 17.06it/s]

In [ ]:
dfs = []
for k in sims.keys():
    results = sims[k]
    results_trimmed = [(1 - r[1], r[2]/(k*k), k) for r in results]
    df = pd.DataFrame(results_trimmed, columns=["p", "ag frac", "L"])
    dfs.append(df)
    
df = pd.concat(dfs)

In [ ]:
sns.set(font_scale=2) 
fig, ax = plt.subplots(1, figsize=(18,6))
sns.lineplot(data=df, x="p", y="ag frac", hue="L", palette="jet", legend="full", ax=ax)
plt.title("Giant Aggregate Sizes", fontsize=24)
plt.show()